In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
from trl import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from trl import ChatGenerationMixin, ChatGeneration, ChatModelOutput

In [ ]:
from huggingface_hub import login

login(token="hf_XXXXXXXXXXXXXXXXXXXXXX")  # Replace with your Hugging Face token


In [ ]:
"""
This script loads a pre-trained Llama-3.2 model, applies a fine-tuned model for NER tagging, and saves the merged model.

Note: Use the appropriate model names and paths for your specific use case.
like check the name of the folders in the huggingface hub
also folder that is created here.
"""
base_model = "meta-llama/Llama-3.2-1B"
new_model = "llama-3-1b-finetuned-ner-tag"

tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()
model.save_pretrained(new_model)
